In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import seaborn as sns
import mplfinance as mpf
import matplotlib.pyplot as plt
import plotly.express as px
from pypfopt import plotting
import math

In [5]:
tickers = ['CL2.PA', 'XS2D.L', 'LCWL.L', 'SSAC.L', 'WSML.L', 'MDBUA.SW']
start_date = '2018-04-01'
end_date = '2023-02-27'
t = 1
n_sims = 10 ** 5

In [17]:
df = yf.download(tickers, start=start_date, end=end_date, adjusted=True)['Adj Close']

[*********************100%***********************]  6 of 6 completed


In [18]:
df = df.dropna()
df = df.rename(columns={"CL2.PA": "MSCI_lev", 
                    "LCWL.L": "MSCI_World",
                    "MDBUA.SW": "Bonds",
                    "SSAC.L": "MSCI_ACWI ",
                    "WSML.L": "Small_cap",
                    "XS2D.L": "S&P_lev"})

In [23]:
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

# Read in price data

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Optimize for maximal Sharpe ratio
'''
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)
'''
# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)


Expected annual return: 10.5%
Annual volatility: 17.2%
Sharpe Ratio: 0.49


(0.10504117448299276, 0.17226623574922273, 0.4936613034651305)

In [24]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

#weights = pd.read_csv('weights.csv')

latest_prices = get_latest_prices(df)

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'MSCI_World': 85, 'Bonds': 1}
Funds remaining: $0.45


/home/alessandro/.local/lib/python3.10/site-packages/pypfopt/discrete_allocation.py:210: RuntimeWarning: invalid value encountered in divide
  current_weights /= current_weights.sum()
